In [1]:
import pandas as pd
import json


df = pd.read_excel("validate.xlsx")
df = df
print(df.head())

import analysisrag
import anaylysis

for i in range(len(df)):
    code = df.iloc[i]["code"]
    try: 
        vulnerabilities = analysisrag.get_vulnerabilities(code)
        df.at[i, "vulnerabilities"] = json.dumps(vulnerabilities)
    except Exception as e:
        print("error", e)
        df.at[i, "vulnerabilities"] = "nan"
    print("vulnerabilities!!!!", vulnerabilities)


                                                code  \
0  // SPDX-License-Identifier: MIT\npragma solidi...   
1  // SPDX-License-Identifier: MIT\npragma solidi...   
2  // SPDX-License-Identifier: MIT\npragma solidi...   
3  // SPDX-License-Identifier: MIT\npragma solidi...   
4  // SPDX-License-Identifier: MIT\npragma solidi...   

                                             contain  \
0  [{\n    "line": "13",\n    "token": "payable(m...   
1                                                 []   
2  [{\n    "line": "8",\n    "token": "balances[m...   
3  [\n  {\n    "line": "14",\n    "token": "Data ...   
4  [\n  {\n    "line": "10",\n    "token": "tx.or...   

                                                miss  
0                                                 []  
1  [{\n    "line": "12",\n    "token": "payable(m...  
2                                                 []  
3                                                 []  
4                                                 [

In [21]:
from nltk.translate.bleu_score import sentence_bleu

def compare_strings(str1, str2):
    """
    Compare two strings using BLEU score.
    """
    reference = [str1.split()]
    candidate = str2.split()
    score = sentence_bleu(reference, candidate)
    return score

In [ ]:
hits = 0
count = 0

BLUE_MODE = True


for i in range(len(df)):
    vulnerabilities = df.iloc[i]["vulnerabilities"]
    contain = df.iloc[i]["contain"]
    miss = df.iloc[i]["miss"]

    if vulnerabilities == "nan":
        print("vulnerabilities is nan")
        continue

    # Filter vulnerabilities with high or medium severity
    vulnerabilities = json.loads(vulnerabilities)
    vulnerabilities = [v for v in vulnerabilities if v["severity"] in ["high", "medium"]]

    problems = [v["problem"] for v in vulnerabilities]
    lines = [v["line"] for v in vulnerabilities]
    tokens = [v["token"] for v in vulnerabilities]
    explanations = [v["explanation"] for v in vulnerabilities]
    migrations = [v["migration"] for v in vulnerabilities]

    # Process missed vulnerabilities
    for entry in json.loads(miss):
        if BLUE_MODE:
            max_explanation_score = max( compare_strings(entry["explanation"], exp) for exp in explanations ) if explanations else 0
            max_migration_score = max( compare_strings(entry["migration"], mig) for mig in migrations ) if migrations else 0
            hits += 2 - max_migration_score - max_explanation_score
        count += 3
        hits += 1 if entry["line"] not in lines else 0
        hits += 1 if entry["problem"] not in problems else 0
        hits += 1 if entry["token"] not in tokens else 0

    # Process contained vulnerabilities
    for entry in json.loads(contain):
        if BLUE_MODE:
            count += 2
            max_explanation_score = max( compare_strings(entry["explanation"], exp) for exp in explanations ) if explanations else 0
            max_migration_score = max( compare_strings(entry["migration"], mig) for mig in migrations ) if migrations else 0
            hits += max_migration_score + max_explanation_score
        count += 3
        hits += 1 if entry["line"] in lines else 0
        hits += 1 if entry["problem"] in problems else 0
        hits += 1 if entry["token"] in tokens else 0

accuracy = hits / count if count > 0 else 0
print("accuracy:", accuracy)


0.2425056620207466
2.0
4.046237560344649e-155
0.08700223397019134
7.7994577434635e-155
accuracy: 0.36215251721699737


d:\ProgramFiles\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\ProgramFiles\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\ProgramFiles\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

In [35]:
for i in df["vulnerabilities"]:
    print(i)

[{"line": "16", "token": "credit[msg.sender] += msg.value;", "problem": "Potential Overflow Vulnerability", "severity": "medium", "explanation": "The addition of msg.value to credit[msg.sender] could exceed the storage limit for uint, leading to an overflow. Although Solidity >=0.8.0 has built-in overflow checking, developers should still be cautious while handling external inputs.", "migration": "Ensure that any amounts added to credit are safe and do not cause overflow, or check that credit[msg.sender] + msg.value does not exceed type(uint).max."}, {"line": "20", "token": "require(payable(msg.sender).call{value: amount} \"\");", "problem": "Reentrancy Vulnerability", "severity": "high", "explanation": "The contract calls an external function before updating its state (credit[msg.sender]), which can be exploited through reentrancy attacks, allowing malicious contracts to withdraw funds repeatedly.", "migration": "Use the Checks-Effects-Interactions pattern by updating the internal sta